## UKCP18 error checking

- Author: Sam Hardy
- Check that the csv files produced by the code in this repository contain the correct number of values for each month

In [19]:
import pandas as pd
import os

### Check for any missing data for the water companies

- Read in data from csv file 
- Generate a list of all expected year-month combinations
- Months with fewer than 13 rows (indicates CEDA connection issues), more than 13 rows (double counted) or no data (also indicates connection issues)
- Reformat the `group_counts` index to pad single numbers (1,2,etc) with a leading zero
- Loop over all ensemble members and year-month combinations

In [27]:
def error_check_ukcp18_data(input_file_path: str, 
                            proj_id: int,
                            member_id: str,
                            output_var: str,
                            FirstTime: bool) -> pd.DataFrame:
    """ 
    Check for missing, incomplete or over-counted year-month combinations in output from UKCP18 hourly rainfall processing code.
    Return a Pandas DataFrame containing all the year-month combinations for a single ensemble member (1,4,5,...,15) and time slice.  
    """

    df = pd.read_csv(input_file_path)
    print(f"Working on {output_var} for ensemble member {member_id:02d}...")

    if FirstTime:
        if proj_id == 1:
            start_year, start_month = 1980, 12
            end_year, end_month = 2000, 11 
        elif proj_id == 2:
            start_year, start_month = 2020, 12 #2000, 12 
            end_year, end_month = 2040, 11 #2020, 11
        elif proj_id == 3: 
            start_year, start_month = 2060, 12 #2040, 12 
            end_year, end_month = 2080, 11 #2060, 11

        expected_year_months = pd.date_range(
            start=f"{start_year}-{start_month:02d}",
            end=f"{end_year}-{end_month:02d}",
            freq="MS"
        ).strftime("%Y-%m").tolist()
    # the text file we input here contains the dates we want to check against - i.e. once we've run the error processing on the original 20-year set, the number of dates will be much smaller (hopefully zero)
    else:
        primary_txt_path = f'/mnt/metdata/2024s1475/UKCP18_Processing_2024/precip_profiles/proj{proj_id}/output_mem{member_id:02d}/v1/proj{proj_id}_{member_id:02d}_error_months_v1.txt'
        secondary_txt_path = f'/mnt/metdata/2024s1475/UKCP18_Processing_2024/precip_profiles/proj{proj_id}/output_mem{member_id:02d}/proj{proj_id}_{member_id:02d}_error_months_v1.txt'
        try:
            print(f'Looking for input file in {primary_txt_path}')
            dates_df = pd.read_csv(primary_txt_path, names=['date_str'], dtype=str) # STANDARD error_month text file written out by code 5 cells after this one 
        except FileNotFoundError:
            print(f'File not found in primary location. Trying in {secondary_txt_path}...')
            if os.path.exists(secondary_txt_path):
                dates_df = pd.read_csv(secondary_txt_path, names=['date_str'], dtype=str)
            else:
                raise FileNotFoundError(f'File not found in either primary or secondary location.')

        expected_year_months = pd.to_datetime(dates_df['date_str'],format="%Y%m").dt.strftime("%Y-%m").tolist()

    df['Year-Month'] = df['Year'].astype(str) + '-' + df['Month'].astype(int).apply(lambda x: f'{x:02d}')
    group_counts = df.groupby('Year-Month').size()
    group_counts.index = group_counts.index.map(lambda x: f"{x.split('-')[0]}-{x.split('-')[1].zfill(2)}")

    # identify errors in the processing 
    missing_months = sorted(set(expected_year_months) - set(group_counts.index))
    incomplete_months = group_counts[group_counts < 13].index.to_list()
    overcounted_months = group_counts[group_counts > 13].index.to_list()

    # flag consecutive months where values for all 13 water companies are the same 
    copied_months = []
    previous_values = None
    grouped = df.groupby('Year-Month')

    # account for different column names for different functions 
    if output_var == 'Total_rainfall':
        value = 'Mean total rainfall'
    elif output_var == 'Dry_days_counts':
        value = 'Mean dry day counts'
    elif output_var == 'Rainfall_bin_counts_1h' or output_var == 'Rainfall_bin_counts_3h' or output_var == 'Rainfall_bin_counts_6h':
        value = 'Bin counts'

    for year_month, group in grouped:
        group_sorted = group.sort_values(by='WCID')
        current_values = tuple(group_sorted[value])

        if previous_values == current_values:
            copied_months.append(year_month)

        previous_values = current_values    

    # combine all months into a single df 
    errors = []
    for value in missing_months:
        errors.append({"Ensemble Member": str(member_id).zfill(2), "Year-Month": value, "Function": output_var, "Issue": "Missing"})
    for value in incomplete_months:
        errors.append({"Ensemble Member": str(member_id).zfill(2), "Year-Month": value, "Function": output_var, "Issue": "Incomplete"})
    for value in overcounted_months:
        errors.append({"Ensemble Member": str(member_id).zfill(2), "Year-Month": value, "Function": output_var, "Issue": "Over-counted"})
    for value in copied_months:
        errors.append({"Ensemble Member": str(member_id).zfill(2), "Year-Month": value, "Function": output_var, "Issue": "Copied"})

    errors_df = pd.DataFrame(errors)

    # Print summary to the console
    if errors:
        print(f"Errors found for ens{member_id:02d}. Details saved to csv file.")
        print(errors_df)
    else:
        print(f"No anomalies found for ens{member_id:02d}. The dataset is complete and consistent.")

    return errors_df

### Loop over all ensemble members and functions (`Total_rainfall`, `Dry_days`, `get_bin_counts`)

- Output to a single csv file for each ensemble member 
- Deal with duplicate values so that we only output one row per year-month combination
- TODO: build in functionality to account for running the code for the first time (`v1`, etc)

In [3]:
proj_ids = [3] # 2021-2040 time slice
#variables = ['Total_rainfall','Dry_days_counts','Rainfall_bin_counts_1h','Rainfall_bin_counts_3h','Rainfall_bin_counts_6h']
variables = ['Rainfall_bin_counts_1h','Rainfall_bin_counts_3h','Rainfall_bin_counts_6h']
#member_ids = [6,7,8,9,10,11,12,13,15]
member_ids = [6,8,10,12]
#member_ids = [4,6,8,10,12,15]
first_time = False

### Modify slightly when running for the first time or again for error processing

In [ ]:
if first_time:
    for proj_id in proj_ids:
        for id in member_ids:
            all_errors = pd.DataFrame()
            for var in variables:
                input_file_path = f'/mnt/metdata/2024s1475/UKCP18_Processing_2024/precip_profiles/proj{proj_id}/output_mem{id:02d}/{var}_ens{id:02d}_proj{proj_id}.csv'
                monthly_calc_errors = error_check_ukcp18_data(input_file_path, 
                                                              proj_id, 
                                                              id,
                                                              var,
                                                              FirstTime=first_time)
                all_errors = pd.concat([all_errors, monthly_calc_errors], ignore_index=True)

            # remove any duplicate rows 
            if all_errors.empty:
                print(f'All months completed successfully for ensemble member {id}!')
            else:
                duplicates_removed = (
                    all_errors.groupby('Year-Month')
                    .agg({
                        'Issue': lambda x: ', '.join(sorted(set(x))),
                        'Function': lambda x: ', '.join(sorted(set(x)))
                        })
                    .reset_index()
                )

                output_file_path = f'/mnt/metdata/2024s1475/UKCP18_Processing_2024/precip_profiles/proj{proj_id}/output_mem{id:02d}/error_checking_proj{proj_id}_ens{id:02d}.csv'
                duplicates_removed.to_csv(output_file_path, index=False)
                print(f'Combined error report saved to {output_file_path}.')
else:
    for proj_id in proj_ids:
        for id in member_ids:
            all_errors = pd.DataFrame()
            for var in variables:
                input_file_path = f'/mnt/metdata/2024s1475/UKCP18_Processing_2024/precip_profiles/proj{proj_id}/output_mem{id:02d}/v1/{var}_ens{id:02d}_proj{proj_id}.csv'
                monthly_calc_errors = error_check_ukcp18_data(input_file_path, 
                                                              proj_id, 
                                                              id, 
                                                              var,
                                                              FirstTime=first_time)
                all_errors = pd.concat([all_errors, monthly_calc_errors], ignore_index=True)

            # remove any duplicate rows 
            if all_errors.empty:
                print(f'All months completed successfully for ensemble member {id}!')
            else:
                duplicates_removed = (
                    all_errors.groupby('Year-Month')
                    .agg({
                        'Issue': lambda x: ', '.join(sorted(set(x))),
                        'Function': lambda x: ', '.join(sorted(set(x)))
                        })
                    .reset_index()
                )

                output_file_path = f'/mnt/metdata/2024s1475/UKCP18_Processing_2024/precip_profiles/proj{proj_id}/output_mem{id:02d}/v1/error_checking_proj{proj_id}_ens{id:02d}.csv'
                duplicates_removed.to_csv(output_file_path, index=False)
                print(f'Combined error report saved to {output_file_path}.')

Working on Rainfall_bin_counts_1h for ensemble member 06...
Errors found for ens06. Details saved to csv file.
   Ensemble Member Year-Month                Function       Issue
0               06    2061-03  Rainfall_bin_counts_1h     Missing
1               06    2062-03  Rainfall_bin_counts_1h     Missing
2               06    2063-09  Rainfall_bin_counts_1h     Missing
3               06    2064-11  Rainfall_bin_counts_1h     Missing
4               06    2064-12  Rainfall_bin_counts_1h     Missing
..             ...        ...                     ...         ...
92              06    2079-12  Rainfall_bin_counts_1h     Missing
93              06    2062-09  Rainfall_bin_counts_1h  Incomplete
94              06    2062-13  Rainfall_bin_counts_1h  Incomplete
95              06    2064-09  Rainfall_bin_counts_1h  Incomplete
96              06    2064-13  Rainfall_bin_counts_1h  Incomplete

[97 rows x 4 columns]
Working on Rainfall_bin_counts_3h for ensemble member 06...
Errors found f

### Create single-column text files for each ensemble member 

- Extract the list of dates 
- Change the format slightly 
- Output list can be fed directly into the `ukcp18_wrapper_script.sh` shell script to rerun the months that we've identified

In [17]:
proj_ids = [3] # 2021-2040 time slice
variables = ['Rainfall_bin_counts_1h','Rainfall_bin_counts_3h','Rainfall_bin_counts_6h']
member_ids = [1,4,6,8,10,12]
member_ids = [5,7,9,11,13,15]
first_time = True

### The input to the code below still needs to be automated 

- I am currently manually editing the cell above to change the values of `proj_id`, `member_ids`, etc
- This part should be joined with the processing function above to create a more streamlined workflow 

In [18]:
if first_time:
    for proj_id in proj_ids:
        for id in member_ids:
            input_csv_path = f'/mnt/metdata/2024s1475/UKCP18_Processing_2024/precip_profiles/proj{proj_id}/output_mem{id:02d}/error_checking_proj{proj_id}_ens{id:02d}.csv'
            dates_for_processing = pd.read_csv(input_csv_path)
            dates_for_processing['Year-Month'] = dates_for_processing['Year-Month'].str.replace("-","")
            error_months = dates_for_processing['Year-Month'].unique()
            output_txt_path = f'/mnt/metdata/2024s1475/UKCP18_Processing_2024/precip_profiles/proj{proj_id}/output_mem{id:02d}/proj{proj_id}_{id:02d}_error_months_v1.txt'
            with open(output_txt_path, 'w') as f:
                f.write('\n'.join(error_months))
else:
    for proj_id in proj_ids:
        for id in member_ids:
            input_csv_path = f'/mnt/metdata/2024s1475/UKCP18_Processing_2024/precip_profiles/proj{proj_id}/output_mem{id:02d}/v1/error_checking_proj{proj_id}_ens{id:02d}.csv'
            dates_for_processing = pd.read_csv(input_csv_path)
            dates_for_processing['Year-Month'] = dates_for_processing['Year-Month'].str.replace("-","")
            error_months = dates_for_processing['Year-Month'].unique()
            output_txt_path = f'/mnt/metdata/2024s1475/UKCP18_Processing_2024/precip_profiles/proj{proj_id}/output_mem{id:02d}/v1/proj{proj_id}_{id:02d}_error_months_v1.txt'
            with open(output_txt_path, 'w') as f:
                f.write('\n'.join(error_months))

### Filter out specific years that we know we don't want to process again

- This part of the code is only relevant if we need to process 2020 or 2060 
- Kay has messaged the Met Office to see whether they have made any progress in identifying the source of the missing data
- Calling the function is currently commented out

In [ ]:
def filter_specific_years_ukcp18_processing(ignored_years: list[str],
                                            proj_id: str,
                                            member_id: str,
                                            input_file_path: str) -> list[str]:
    """ 
    Filter specific years that we know we don't want to process any further 
    """

    dates_df = pd.read_csv(input_file_path, names=['date_str'], dtype=str)

    # format the dates to add a hyphen between the YYYY and MM parts of the string 
    expected_year_months = pd.to_datetime(dates_df['date_str'],format="%Y%m").dt.strftime("%Y-%m").tolist()
    # create a DataFrame from the list and name the column "date"
    df = pd.DataFrame(expected_year_months,columns=["date"])

    # parse the dates and retrieve the year from each 
    df["year"] = pd.to_datetime(df["date"]).dt.year
    # filter the df to remove any years that match our list above 
    filtered_df = df[~df["year"].isin(ignored_years)]
    filtered_df["date"] = filtered_df["date"].str.replace("-","")
    filtered_dates = filtered_df['date'].unique().tolist()

    # convert any elements back to a list 
    with open(f'proj{proj_id}_{member_id:02d}_error_months_v1_filtered.txt', 'w') as f:
        f.write('\n'.join(filtered_dates))

# ignored_years = [2020, 2060]
# proj_id=3
# for proj_id in proj_ids:
#     for id in member_ids:
#         input_txt_path = f'./proj{proj_id}_{id:02d}_error_months_v1.txt'
#         filter_specific_years_ukcp18_processing(ignored_years,
#                                                 proj_id,
#                                                 id,
#                                                 input_txt_path)